To-do: rewrite this notebook to be compatiable with clean.ipynb

## Setup

In [ ]:
from util.plot import plot_interactive_lines

## KLEMS 2019 

In [ ]:
temp = (
    df_NA.query('code=="TOT"')
    .eval("GO_per_EMP = GO / EMP")
    .pivot("year", "country", "GO_per_EMP").pct_change()
)

In [ ]:
plot_interactive_lines(
    data=temp.stack().rename('growth').reset_index().query('country not in ["BG"]'),
    x="year",y="growth",hue="country", point_size=15, line_size=1.5,
)

## KLEMS 2013 

### Setup

In [ ]:
file = pd.ExcelFile('data/JPN_wk_may_2013.xlsx') # usa_wk_apr_2013

In [ ]:
file.sheet_names

In [ ]:
# print variables
prints(file.parse('Notes', usecols=[0,1])
              .dropna(how="all")
              .set_axis(["Variable", "Description"], axis=1)
      )

In [ ]:
df = file.parse('DATA')

In [ ]:
df = (df.rename(columns = lambda x: pd.to_numeric(x.replace('_',''), 
                                                       errors='ignore'))
        )

In [ ]:
# industry code
(df.code + ' -> ' + df.desc).unique()

In [ ]:
ind2d = df.code[df.code.str.contains("^[A-Z]")].unique()

### Test

In [ ]:
# labor share
var = ["VA", "LAB"]
ind = "TOT"
(df.query("Variable in @var & code in @ind")
      .drop(["desc","code"],axis=1).set_index("Variable").T
      .eval("labor_share = LAB / VA")
).plot(y="labor_share");

In [ ]:
# labor share by 2-digit industries
temp = (df.query("Variable in @var & code in @ind2d").drop(["desc"],axis=1)
      .groupby("code").apply(lambda x: x.set_index("Variable").T.eval('LAB/VA')).stack()
      .reset_index().set_axis(["IND","YEAR","LABOR_SHARE"], axis=1)
)
selection = alt.selection_multi(fields=['IND'], bind='legend')
alt.Chart(temp).mark_line().encode(
    x='YEAR',
    y='LABOR_SHARE',
    color='IND',
    tooltip=['IND',],
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
).add_selection(selection)

In [ ]:
# calculate labor productivity on Value-added
var = ["VA", "H_EMP"]
ind = "TOT"
(df.query("Variable in @var & code in @ind")
      .drop(["desc","code"],axis=1).set_index("Variable").T
      .eval("labor_P = VA / H_EMP")
      .pct_change()
).plot(y="labor_P");

In [ ]:
# plot TFP on Value-added in data
var = ["TFPva_I"]
(df.query("Variable in @var & code in @ind")
      .drop(["desc","code"],axis=1).set_index("Variable").T
      .pct_change()
).plot();